In [49]:
# def get_next_NYSE_date(my_date, dates_NYSE_past_10yr):

#   if my_date in dates_NYSE_past_10yr:
#     idx_date = dates_NYSE_past_10yr.get_loc(my_date)

#     if idx_date >= len(dates_NYSE_past_10yr) - 1:
#       print(f'Next NYSE trading date after {my_date} is not a trading date in dates_NYSE_past_10yr')
#       print(f'dates_NYSE_past_10yr:\n{dates_NYSE_past_10yr}')

#     else:
#       date_next = dates_NYSE_past_10yr[idx_date + 1].strftime('%Y-%m-%d')  

#   else:
#     msg_stop_date_NYSE = f'ERROR: my_date: {my_date} is not a trading date in dates_NYSE_past_10yr\ndates_NYSE_past_10yr:\n{dates_NYSE_past_10yr}'
#     raise SystemExit(msg_stop_date_NYSE )

#   return date_next  

In [50]:
# # create empty DataFrame df and append row
# my_cols = ['date', 'days_lookback', 'syms_freq', 'symbols']
# # Creating Empty DataFrame and Storing it in variable df
# df_model_top_picks = pd.DataFrame(columns=my_cols)
# df_model_top_picks

In [51]:
import pandas as pd
from myUtils import pickle_load, pickle_dump
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_colwidth', 30)
pd.set_option('display.width', 900)

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"

fp_df_picks  = f'df_picks'
fp_df_model_top_picks = f'df_model_top_picks'

In [52]:
df = pickle_load(path_data_dump, fp_df_picks)
df = df.drop_duplicates(subset=['date_end_df_train', 'max_days_lookbacks', 'days_lookbacks'], keep='last')
df = df.sort_values(by=['date_end_df_train', 'max_days_lookbacks', 'days_lookbacks'], ascending=False)
df = df.reset_index(drop=True)
pickle_dump(df, path_data_dump, fp_df_picks)
print(f'len(df): {len(df)}')
print(df.head())

len(df): 38
  date_end_df_train  max_days_lookbacks     days_lookbacks sym_freq_15 sym_freq_14 sym_freq_13 sym_freq_12 sym_freq_11 sym_freq_10 sym_freq_9 sym_freq_8      sym_freq_7       sym_freq_6                     sym_freq_5                     sym_freq_4                     sym_freq_3 sym_freq_2
0        2023-04-11                 120      [30, 60, 120]          []          []          []          []          []          []         []    ['SHV']        ['FTSM']  ['AMPH', 'ELF']       ['META', 'NVDA', 'PDFS']  ['ACLS', 'AU', 'MLR', 'NVO...  ['ANET', 'AQN', 'BTC-USD',...         []
1        2023-04-11                 120  [15, 30, 60, 120]          []          []          []          []          []          []    ['SHV']         []  ['AU', 'FTSM']  ['AMPH', 'ELF']       ['META', 'NVDA', 'PDFS']  ['ACLS', 'CNK', 'MLR', 'NV...         ['ANET', 'AQN', 'AZN']         []
2        2023-04-10                 120      [30, 60, 120]          []          []          []          []          []

In [53]:
from itertools import product
from ast import literal_eval

my_cols = ['sym_freq_12', 'sym_freq_9', 'sym_freq_8']
my_days_lookbacks = [[30, 60, 120], [15, 30, 60, 120]]

my_dates = []
for my_days_lookback in my_days_lookbacks:
  my_row = (df['days_lookbacks'] == str(my_days_lookback))
  l_dates = df.loc[my_row, 'date_end_df_train'].tolist()     
  my_dates.append(l_dates)

my_dates = [val for sublist in my_dates for val in sublist]
my_dates = sorted(list(set(my_dates)), reverse=True)

d_lbk_freq = list(product(my_dates, my_days_lookbacks, my_cols))

picks = []
prev_date = ''
# print(f'Model predicts the best performing symbols are:\n[30, 60, 120]        sym_freq_9\n[15, 30, 60, 120]    sym_freq_12\n[30, 60, 120]        sym_freq_8')
print(f'Model predicts the best performing symbols are:')
print(f'1st    {"[30, 60, 120]":<20}{"sym_freq_9":<14}{"days_eval = 4":<20}')
print(f'2nd    {"[15, 30, 60, 120]":<20}{"sym_freq_12":<14}{"days_eval = 5":<20}')
print(f'3rd    {"[30, 60, 120]":<20}{"sym_freq_8":<14}{"days_eval = 4":<20}')
for i, item in enumerate(d_lbk_freq):
  _date, my_days_lookback, my_col = item[0], item[1], item[2]
  my_row = (df['date_end_df_train'] == _date) & (df['days_lookbacks'] == str(my_days_lookback))  
  list_in_df = df.loc[my_row, my_col]
  my_str = list_in_df.tolist()[0]
  my_list = literal_eval(my_str)
  blank = ' '
  if _date != prev_date:
    if prev_date == '':
      print('\nModel Results:')  
      print(f'{i:<6}{_date:<14}{str(my_days_lookback):<21}{my_col:<11}: {my_list}')
    else:
      print(f'\n{i:<6}{_date:<14}{str(my_days_lookback):<21}{my_col:<11}: {my_list}') 
  else:
    print(f'{i:<6}{blank:<14}{str(my_days_lookback):<21}{my_col:<11}: {my_list}')

  picks.append((i, my_list))
  prev_date = _date

Model predicts the best performing symbols are:
1st    [30, 60, 120]       sym_freq_9    days_eval = 4       
2nd    [15, 30, 60, 120]   sym_freq_12   days_eval = 5       
3rd    [30, 60, 120]       sym_freq_8    days_eval = 4       

Model Results:
0     2023-04-11    [30, 60, 120]        sym_freq_12: []
1                   [30, 60, 120]        sym_freq_9 : []
2                   [30, 60, 120]        sym_freq_8 : ['SHV']
3                   [15, 30, 60, 120]    sym_freq_12: []
4                   [15, 30, 60, 120]    sym_freq_9 : ['SHV']
5                   [15, 30, 60, 120]    sym_freq_8 : []

6     2023-04-10    [30, 60, 120]        sym_freq_12: []
7                   [30, 60, 120]        sym_freq_9 : []
8                   [30, 60, 120]        sym_freq_8 : ['SHV']
9                   [15, 30, 60, 120]    sym_freq_12: []
10                  [15, 30, 60, 120]    sym_freq_9 : ['SHV']
11                  [15, 30, 60, 120]    sym_freq_8 : []

12    2023-04-06    [30, 60, 120]        sym

In [54]:
model_top_picks = [([30, 60, 120], 'sym_freq_9'), ([15, 30, 60, 120], 'sym_freq_12'), ([30, 60, 120], 'sym_freq_8')]

In [55]:
df_model_top_picks = pickle_load(path_data_dump, fp_df_model_top_picks)

z_items   = zip(d_lbk_freq, picks)
for i, item in enumerate(z_items):
  _date, _days_lookback, _sym_freq, _syms = item[0][0], item[0][1], item[0][2], item[1][1]
  print(i, _date, _days_lookback, _sym_freq, _syms)
  dlb_sym_freq = (_days_lookback, _sym_freq)
  if dlb_sym_freq in model_top_picks and _syms:
    row_add = [_date, _days_lookback, _sym_freq, _syms]
    df_model_top_picks.loc[len(df_model_top_picks)] = row_add
    print(f'added row to df_model_top_picks:\n{row_add}\n')


0 2023-04-11 [30, 60, 120] sym_freq_12 []
1 2023-04-11 [30, 60, 120] sym_freq_9 []
2 2023-04-11 [30, 60, 120] sym_freq_8 ['SHV']
added row to df_model_top_picks:
['2023-04-11', [30, 60, 120], 'sym_freq_8', ['SHV']]

3 2023-04-11 [15, 30, 60, 120] sym_freq_12 []
4 2023-04-11 [15, 30, 60, 120] sym_freq_9 ['SHV']
5 2023-04-11 [15, 30, 60, 120] sym_freq_8 []
6 2023-04-10 [30, 60, 120] sym_freq_12 []
7 2023-04-10 [30, 60, 120] sym_freq_9 []
8 2023-04-10 [30, 60, 120] sym_freq_8 ['SHV']
added row to df_model_top_picks:
['2023-04-10', [30, 60, 120], 'sym_freq_8', ['SHV']]

9 2023-04-10 [15, 30, 60, 120] sym_freq_12 []
10 2023-04-10 [15, 30, 60, 120] sym_freq_9 ['SHV']
11 2023-04-10 [15, 30, 60, 120] sym_freq_8 []
12 2023-04-06 [30, 60, 120] sym_freq_12 []
13 2023-04-06 [30, 60, 120] sym_freq_9 []
14 2023-04-06 [30, 60, 120] sym_freq_8 ['SHV']
added row to df_model_top_picks:
['2023-04-06', [30, 60, 120], 'sym_freq_8', ['SHV']]

15 2023-04-06 [15, 30, 60, 120] sym_freq_12 []
16 2023-04-06 [15,

In [56]:
df_model_top_picks = df_model_top_picks.astype(str)
df_model_top_picks = df_model_top_picks.drop_duplicates(keep='last')
df_model_top_picks = df_model_top_picks.sort_values(by=['date', 'days_lookback', 'syms_freq'], ascending=False)
df_model_top_picks = df_model_top_picks.reset_index(drop=True)
pickle_dump(df_model_top_picks, path_data_dump, fp_df_model_top_picks)
print(f'len(df_model_top_picks): {len(df_model_top_picks)}')
print(f'{fp_df_model_top_picks}:\n{df_model_top_picks}')

len(df_model_top_picks): 24
df_model_top_picks:
          date  days_lookback   syms_freq          symbols
0   2023-04-11  [30, 60, 120]  sym_freq_8          ['SHV']
1   2023-04-10  [30, 60, 120]  sym_freq_8          ['SHV']
2   2023-04-06  [30, 60, 120]  sym_freq_8          ['SHV']
3   2023-04-05  [30, 60, 120]  sym_freq_9          ['SHV']
4   2023-04-04  [30, 60, 120]  sym_freq_8  ['NVDA', 'SHV']
5   2023-04-03  [30, 60, 120]  sym_freq_8   ['GE', 'NVDA']
6   2023-03-31  [30, 60, 120]  sym_freq_8         ['NVDA']
7   2023-03-30  [30, 60, 120]  sym_freq_9          ['SHV']
8   2023-03-30  [30, 60, 120]  sym_freq_8         ['NVDA']
9   2023-03-29  [30, 60, 120]  sym_freq_9          ['SHV']
10  2023-03-28  [30, 60, 120]  sym_freq_9          ['SHV']
11  2023-03-27  [30, 60, 120]  sym_freq_9          ['SHV']
12  2023-03-27  [30, 60, 120]  sym_freq_8           ['GE']
13  2023-03-24  [30, 60, 120]  sym_freq_8  ['NVDA', 'SHV']
14  2023-03-23  [30, 60, 120]  sym_freq_9          ['SHV']
15  2023

In [57]:
df_mp = df_model_top_picks.copy()
df_close = pickle_load(path_data_dump, 'df_close_clean')

In [58]:
import pandas_market_calendars as mcal
import datetime
today = datetime.date.today()
yr_10 = datetime.timedelta(days = 365*10)

nyse = mcal.get_calendar('NYSE')
dates_NYSE_past_10yr = nyse.valid_days(start_date=today - yr_10, end_date=today)

In [59]:
sorted_df_close_idx = sorted(df_close.index)
print(f'len(sorted_df_close_idx: {len(sorted_df_close_idx)}')
print(f'sorted_df_close_idx[:3]: {sorted_df_close_idx[:3]}')
print(f'sorted_df_close_idx[-3:]: {sorted_df_close_idx[-3:]}\n')

print(f'len(dates_NYSE_past_10yr: {len(dates_NYSE_past_10yr)}')
print(f'dates_NYSE_past_10yr: {dates_NYSE_past_10yr[:3]}')
print(f'dates_NYSE_past_10yr: {dates_NYSE_past_10yr[-3:]}')

len(sorted_df_close_idx: 1500
sorted_df_close_idx[:3]: [Timestamp('2017-04-26 00:00:00'), Timestamp('2017-04-27 00:00:00'), Timestamp('2017-04-28 00:00:00')]
sorted_df_close_idx[-3:]: [Timestamp('2023-04-06 00:00:00'), Timestamp('2023-04-10 00:00:00'), Timestamp('2023-04-11 00:00:00')]

len(dates_NYSE_past_10yr: 2516
dates_NYSE_past_10yr: DatetimeIndex(['2013-04-15 00:00:00+00:00', '2013-04-16 00:00:00+00:00', '2013-04-17 00:00:00+00:00'], dtype='datetime64[ns, UTC]', freq=None)
dates_NYSE_past_10yr: DatetimeIndex(['2023-04-06 00:00:00+00:00', '2023-04-10 00:00:00+00:00', '2023-04-11 00:00:00+00:00'], dtype='datetime64[ns, UTC]', freq=None)


In [60]:
def get_next_date(df_close, date):
    """Get the next date from df_close. date has to in df_close.index.

    Edge case: The function won't return the first date in df_close as the
    next trading date, if date is a trading date just before the first date in df_close.
    """

    len_df_close = len(df_close)
    idx_dates = df_close.index.strftime('%Y-%m-%d')  # date index of df_close    

    if date in idx_dates:
        idx_next_date = idx_dates.get_loc(date) + 1  # get index position of next date

        if idx_next_date <= (len_df_close - 1):  # next date's index is within bound of df_close
            next_date = idx_dates[idx_dates.get_loc(date) + 1]  # get the next date
            return next_date  
        else:  # idx_next_date is out-of_bounds df_close
            print(f"{date} is the last date in df_close. No data for next day")
            return None    
    else:     
        # no data from df_close
        print(f'no data for {date} in df_close')
        return None

In [61]:
def get_symbols_close(my_date, df_close):
  # get closing price for symbols for my_date, i.e. a row of df_close for my_date
  # return symbols' closing price as a series
  if my_date in df_close.index:  
    iloc_my_date = df_close.index.get_loc(my_date)
    print(f'len(df_close): {len(df_close)}, my_date: {my_date}, iloc_my_date: {iloc_my_date}')      
    symbols_close = df_close.iloc[iloc_my_date]    
  else:
    msg_stop = f'ERROR: my_date: {my_date} is not a date in df_close.index\ndf_close.index:\n{df_close.index}'
    raise SystemExit(msg_stop)
  return symbols_close


In [64]:
def get_portf_buy(df_close, date, str_symbols, portf_target):    
    l_syms = literal_eval(str_symbols)  # convert list stored as str back to list
     # array of closing prices corresponding to symbols in l_syms    
    ar_price = df_close.loc[date][l_syms].values  
    sym_cnt = len(l_syms)
    amt_per_sym = portf_target / sym_cnt
    ar_sym_share = np.floor(amt_per_sym / ar_price)
    ar_sym_amt = ar_price * ar_sym_share
    portf_value = sum(ar_sym_amt)
    print(date, l_syms, ar_price, sym_cnt, ar_sym_share, ar_sym_amt, portf_value)
    print(f'{date} {portf_value = }\n')

In [66]:
import numpy as np
from ast import literal_eval
z_date_syms = zip(df_mp.date, df_mp.symbols)
portf_target = 1000  # ideal value of initial portfolio amount 
for _date, _syms in z_date_syms:
    # get_portf_amt(df_close, _date, _syms)
    next_date = get_next_date(df_close, _date)
    print(_date, next_date, _syms)
    if next_date is None:
        print(f'No data for next_date, portf value = None\n')
    else:    
        get_portf_buy(df_close, next_date, _syms, portf_target=1000)
    

2023-04-11 is the last date in df_close. No data for next day
2023-04-11 None ['SHV']
No data for next_date, portf value = None

2023-04-10 2023-04-11 ['SHV']
2023-04-11 ['SHV'] [110.20999908] 1 [9.] [991.88999176] 991.8899917602539
2023-04-11 portf_value = 991.8899917602539

2023-04-06 2023-04-10 ['SHV']
2023-04-10 ['SHV'] [110.19000244] 1 [9.] [991.71002197] 991.7100219726562
2023-04-10 portf_value = 991.7100219726562

2023-04-05 2023-04-06 ['SHV']
2023-04-06 ['SHV'] [110.19999695] 1 [9.] [991.79997253] 991.7999725341797
2023-04-06 portf_value = 991.7999725341797

2023-04-04 2023-04-05 ['NVDA', 'SHV']
2023-04-05 ['NVDA', 'SHV'] [268.80999756 110.19999695] 2 [1. 4.] [268.80999756 440.79998779] 709.6099853515625
2023-04-05 portf_value = 709.6099853515625

2023-04-03 2023-04-04 ['GE', 'NVDA']
2023-04-04 ['GE', 'NVDA'] [ 95.05000305 274.52999878] 2 [5. 1.] [475.25001526 274.52999878] 749.7800140380859
2023-04-04 portf_value = 749.7800140380859

2023-03-31 2023-04-03 ['NVDA']
2023-04-03 [